In [1]:
import os
from shutil import copyfile

import numpy as np

import torch
from torch.optim import lr_scheduler
from torch.autograd import Variable

import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.models as models

# 1. Data Preprocessing

In [2]:
def listdir_fullpath(d):
    return [os.path.join(d, f) for f in os.listdir(d)]

In [3]:
def set_up_folders():
    
    os.rename('Dataset_A/data', 'data')
    
    for old_fname in listdir_fullpath('Dataset_A'):
        new_fname = os.path.join('data', old_fname.split('/')[-1])
        os.rename(old_fname, new_fname)

    os.rmdir('Dataset_A')
    
    for folder in ['train', 'val', 'test']:
        if folder not in os.listdir('data/'):
            os.mkdir(os.path.join('data/', folder))

            if folder in ['train', 'val']:
                os.mkdir(os.path.join('data/', folder, 'cancer'))
                os.mkdir(os.path.join('data/', folder, 'non_cancer'))

if 'Dataset_A' in os.listdir('.'):
    set_up_folders()

In [4]:
label_map = {'0': 'non_cancer', '1': 'cancer'}
added_lbl = []

def set_up_files():
    
    all_fnames = [x for x in os.listdir('data') if x.endswith('.png')]
    
    for folder in ['test', 'val', 'train']:                                  
        with open('data/{}.txt'.format(folder), 'r') as cur_file:            
            for line in cur_file:
                
                cur_prefx = line.split()[0]
                cur_fname = [x for x in all_fnames if x.startswith(cur_prefx)][0]
                cur_opath = os.path.join('data', cur_fname)
                
                if folder == 'test':
                    cur_dpath = os.path.join('data', folder, cur_fname)
                    os.rename(cur_opath, cur_dpath)
                    
                elif folder in ['val', 'train']:
                    if cur_prefx not in added_lbl:
                        cur_label = label_map[line.split()[1]]
                        cur_dpath = os.path.join('data', folder, cur_label, cur_fname)                    
                        os.rename(cur_opath, cur_dpath)
                    
                added_lbl.append(cur_prefx)
                    
if ['train', 'val', 'test'] not in os.listdir('data'):
    set_up_files()